In [1]:
import polars as pl
import numpy as np
import polars_ds as pld

In [ ]:
size = 2000
df = pl.DataFrame({
    "a": np.random.random(size=size),
})
df.head()

In [ ]:
%%timeit
df.select(
    pl.col("a").num.approximate_entropy(m=2, filtering_level = 0.9, parallel=True)
)

In [ ]:

df.select(
    pl.col("test").num.sample_entropy(m=2)
)


In [2]:
size = 5000
df = pl.DataFrame({
    "market_id": range(size),
    "group1": np.random.random(size=size),
    "group2": np.random.random(size=size),
    "category_1": np.random.randint(low=0, high=5, size=size),
    "category_2":np.random.randint(low=0, high=10, size=size)
}).with_columns(
    pl.col("market_id").mod(3)
)
df.head(5)

market_id,group1,group2,category_1,category_2
i64,f64,f64,i64,i64
0,0.863211,0.05146,2,0
1,0.375187,0.841213,0,4
2,0.731748,0.775747,2,2
0,0.366678,0.236327,1,3
1,0.521338,0.728827,3,6


In [3]:
# In segment T-test, chi2 test, F test made easy!
df.group_by("market_id").agg(
    pl.col("group1").stats.ttest_ind(pl.col("group2"), equal_var = True).alias("t-test"),
    pl.col("category_1").stats.chi2(pl.col("category_2")).alias("chi2-test"),
    pl.col("category_1").stats.f_test(pl.col("group1")).alias("f-test")
)

market_id,t-test,chi2-test,f-test
i64,struct[2],struct[2],struct[2]
0,"{-0.678227,0.497675}","{27.55702,0.842609}","{3.304733,0.010454}"
1,"{1.049668,0.293947}","{28.920644,0.792821}","{1.081389,0.364112}"
2,"{0.246265,0.805493}","{27.843576,0.832729}","{0.851298,0.492663}"


In [ ]:
df.select(
    pld.nb_cnt(
        0.1, # distance is always the first argument
        pl.col("val1"), pl.col("val2"), pl.col("val3"), # Columns used as the coordinates in n-d space
        dist = "inf", parallel = True 
    )
)

In [ ]:
df.select(
    pld.haversine(pl.col("val1"), pl.col("val2"), pl.col("val3"), pl.col("val4"))
)

In [ ]:
df.filter(
    pld.query_radius(
        [0,0],
        pl.col("val1"), pl.col("val2"), # pl.col("val3"),
        radius = 10, dist = "h"
    )
)

In [ ]:
import polars_ds  # noqa: F811

print(
    df.with_columns(
        pl.col("id").cast(pl.UInt64).num.knn_ptwise(
            pl.col("val1"), pl.col("val2"), 
            k = 3, dist = "haversine", parallel = True
        ).alias("nearest neighbor ids")
    ).head()
)

In [ ]:
import numpy as np
from scipy import signal
rng = np.random.default_rng()

npoints = 10

noise = rng.standard_normal(npoints)

x = 3 + 2*np.linspace(0, 1, npoints) + noise

df = pl.DataFrame({
    "test": x,
    "entity_id": [1] *  5 + [2] * 5
})
df.head()

In [ ]:
import polars as pl
import polars_ds

df = pl.DataFrame({
    "test": x,
    "entity_id": [1] *  5 + [2] * 5
})
# And is 5x faster than Scipy.signal detrend on larger time series
df.select(
    pl.col("entity_id"),
    pl.col("test").num.detrend().over(pl.col("entity_id")).alias("test_detrended") # linear detrend
)

In [ ]:
%timeit signal.detrend(x)

In [ ]:
%timeit df.select(pl.col("test").num.detrend()).head()

In [ ]:
df.select(pl.col("test").num.detrend2()).head()

In [ ]:
df = pl.DataFrame({
    "c":[[0.1, 0.2], [0.5, 0.5], [-2, 2]],
    "z":[[0.1, 0.2], [0.5, 0.5], [-2, 2]]
})
df.head()

In [ ]:
df = pl.DataFrame({
    "a": list(range(10)) + [None] # Reference column
}).with_columns(
    pl.col("a").stats.rand_int(low=1., high=10, respect_null=True).alias("rand_int"),
    pl.col("a").stats.sample_uniform(low=1., high=3.).alias("uniform"),
    pl.col("a").stats.sample_normal(respect_null=True).alias("normal1"),
    pl.col("a").stats.sample_normal(mean = 2, std = 0.5).alias("normal2"),
    pl.col("a").stats.sample_exp(lam = 1.0).alias("exp"),
    pl.col("a").stats.sample_binomial(n = 10, p = 0.5).alias("binomial"),
    pl.col("a").stats.rand_str(min_size = 1, max_size = 10, respect_null=True).alias("rand_str")
)
df

In [ ]:
df.select(
    pl.col("a").stats.f_stats(pl.col("b"), pl.col("c"))
).item(0,0)

In [ ]:

df.select(
    pl.col("a").stats.f_test(pl.col("b"))
).item(0,0)

In [ ]:
from sklearn.feature_selection import f_regression, f_classif

In [ ]:
f_classif(df["b"].to_numpy().reshape(-1,1), df["a"].to_numpy())

In [ ]:
f_classif(df["c"].to_numpy().reshape(-1,1), df["a"].to_numpy())

In [ ]:
df.select(
    pl.col("a").str.sorensen_dice(pl.col("b"))
)

In [ ]:
df.select(
    pl.col("a").num.list_jaccard(pl.col("b"))
)

In [ ]:
df = pl.DataFrame({
    "a":["a", "b", "c", "c", "c", "c", "c", "c", "c", "c", "c", "c", "c", "c"],
    "b":["a", "b", "c", "d", "d", "d", "d", "d", "d", "d", "d", "d", "d", "d"]
})
df.head()

In [ ]:
df.select(
    pl.concat_str(pl.col("a"), pl.col("b"))
)

In [ ]:
df = pl.DataFrame({
    "a":["a", "b", "c", "c", "c", "c", "c", "c", "c", "c", "c", "c", "c", "c"],
    "b":["a", "b", "c", "d", "d", "d", "d", "d", "d", "d", "d", "d", "d", "d"]
})
vc = pl.col("a").value_counts(parallel=False, sort=True)
to_merge:pl.Expr = (
    vc.filter(
        vc.struct.field("counts") < 3
    ).struct.field("a")
)
df.select(to_merge)

In [ ]:
df.select(
    pl.col("b").str_ext.infer_infreq()
)